## <font color='darkblue'>Preface</font>
([article source](https://levelup.gitconnected.com/python-interfaces-choose-protocols-over-abc-3982e112342e)) <b><font size='3ptx'>Several practical reasons to prefer new shiny Protocols</font></b>

I used a standard Python library [**abc**](https://docs.python.org/3/library/abc.html) to define interfaces for the last 10 years of my career. But recently, I found that relatively new [**Python Protocols**](https://peps.python.org/pep-0544/) are way nicer. People find uses for both technologies. But **I want to convince you to completely jump ships and start using them instead of more traditional techniques**.

## <font color='darkblue'>Python interfaces: what can you use?</font>
<b><font size='3ptx'>Python is somewhat different from other popular languages since there are no interfaces on a language level</font></b>. But there are several library implementations. The [**abc**](https://docs.python.org/3/library/abc.html) package is probably the most popular:

In [1]:
from abc import ABC, abstractmethod

class Animal(ABC):
  @abstractmethod
  def eat(self, food) -> float:
    pass

  @abstractmethod
  def sleep(self, hours) -> float:
    pass

Next, the most frequently mentioned package seems to be [**Zope**](https://pypi.org/project/Zope/):
```python
from zope.interface import Interface

class Animal(Interface):
  def eat(self, food) -> float:
    pass
  def sleep(self, hours) -> float:
    pass
```

[**Zope**](https://pypi.org/project/Zope/) is a web-related library, and its interfaces have a lot of advanced features. Also, there are more custom packages and tutorials on the web on how to make an interface system yourself ([see example](https://realpython.com/python-interface/)).

Finally, there are protocols:

In [2]:
from typing import Protocol

class Animal(Protocol):
  def eat(self, food) -> float:
    ...
    
  def sleep(self, hours) -> float:
    ...

A protocol is a formalization of Python’s “[duck-typing](https://en.wikipedia.org/wiki/Duck_typing)” ideology. There are many great articles on structural typing in Python (<font color='brown'>for example, see [this tutorial](https://trm.io/2017/01/29/structural-subtyping-python)</font>). Protocols and interfaces are different beasts in theory, but a protocol does the job. I had great success replacing [**abc**](https://docs.python.org/3/library/abc.html) with [**Protocols**](https://peps.python.org/pep-0544/) without any downsides.

## <font color='darkblue'>Niche and Comparison</font>
<b><font size='3ptx'>You should know that an interface system will not be localized to a small part of your codebase. After you choose to go with one, you’ll see it everywhere, and it’s going to be hard to change in the future. </font></b>

<b>So I would immediately dismiss any custom implementations or [Zope](https://pypi.org/project/Zope/)</b>. It’s an extra dependency you have to deal with forever: installation, versions, support, and so on. For example, you have to install a [**Mypy plugin**](https://github.com/Shoobx/mypy-zope#plugin-for-mypy-to-support-zopeinterface) to support a `zope.interface` well. Additionally, a new developer in the team might not know this custom package, and you'll have to explain what it is and why you chose it.

The main battle will happen between `abc` and `Protocols`. But if you really want a `zope` vs `Protocols` battle, please read [this detailed analysis](https://glyph.twistedmatrix.com/2021/03/interfaces-and-protocols.html#runtime-support).

### <font color='darkgreen'>Static checking first</font>
<font size='3ptx'><b>The big assumption I’m going to make is that you’re already convinced that static checking is a must</b></font>: you are not going to run the code that fails `pylint/mypy`. <b>Both checkers support `abc` and `Protocols` equally well. Also, just know that both `abc` and `Protocols` allow runtime checking, in case you need it</b>.

### <font color='darkgreen'>Both support explicit syntax</font>
First, note that you still can explicitly inherit from an `abc` and a `Protocol`. Many arguments in a [very good video](https://www.youtube.com/watch?v=xvb5hGLoK0A) (<font color='brown'>and comments</font>) from Arjan revolve around the misconception that you can’t do that with protocols. You totally can:
```python
class Giraffe(Animal):
   ...
```

So in that regard, `abc` and `protocols` could be used the same way. But **`Protocols` give you an extra degree of design freedom by default. You can avoid explicit inheritance but still enjoy full interface checking**:
```python
class Giraffe:  # no base class needed!
  def eat(self, food) -> float:
    return 0.
 
  def sleep(self, hours) -> float:
    return 1.

def feed_animal(animal: Animal):
    ... 

giraffe = Giraffe()
feed_animal(giraffe)
```

This allows you to make an interface for the code you don’t control and loosen the dependencies between modules in your codebase. Whether to choose an implicit or explicit option is a subtle choice decided on a case-by-case basis. A good example in favour of explicit “opt-in” for an interface is described [here](https://glyph.twistedmatrix.com/2021/03/interfaces-and-protocols.html#where-interfaces-work-best-hidden-and-complex-meanings).

<b>Protocols do not force you to opt-in. But you can still establish a company-wide agreement to always explicitly inherit from any protocol.</b>

[**abc**](https://docs.python.org/3/library/abc.html) also support implicit interfaces through the concept of “<b><font color='darkblue'>virtual subclasses</font></b>” . But you have to call register for every implementation:
```python
class Giraffe:  # no base class needed!
  def eat(self, food) -> float:
    return 0.

class Animal(ABC):
  ...

Animal.register(Giraffe)  # achieves the same as implicit Protocol
```

`Procotol` supports implicit and explicit variants without extra syntax and works with `mypy`. Also, [mypy does not support register as of the end of 2022](https://github.com/python/mypy/issues/2922). I'm not sure if we can fully count that in favour of [**abc**](https://docs.python.org/3/library/abc.html).

**`Protocols` allow you to define an interface for a function** (<font color='brown'>not only a class</font>) — see [callback protocols](https://mypy.readthedocs.io/en/stable/protocols.html#callback-protocols). It is a very cool feature that is worthy of a separate post.

### <font color='darkgreen'>Both support default methods (unfortunately)</font>
There is a big downside to both `abc` and `Protocols`. In the real world, many people work in a single codebase. Abstract base classes sometimes tend to acquire default method implementations. This is what it might look like:
```python
class Animal(Protocol):  # the same holds for Animal(ABC):
  def eat(self, food) -> float:
    ...  # this is still abstract
 
  def sleep(self, hours) -> float:
    return 3.
```

<font color='darkred'><b>In that case, they stop being “abstract” and become just base classes</b></font>. Python and static checkers do not catch that. A software design with inheritance is not really the same as a design with interfaces. I would love Python to separate them on a language level, but it is unlikely to happen. Implicit protocols have an advantage here since they allow you to avoid messy inheritance altogether.

### <font color='darkgreen'>Protocols are shorter</font>
Last but not least, you can count the number of lines of code you need to define an interface. With `abc`, you must have an `abstractmethod` decorator for every method. But with `Protocols` without runtime checking, you don't have to use any decorators at all. So here, Protocols win hands down.

## <font color='darkblue'>Conclusion</font>
Let’s add up the scores:
![comparison table](https://miro.medium.com/max/1400/1*Em8M0KwmDHXsliHOaBwJ7Q.webp)

Hopefully, I’m not missing anything huge in this analysis. Looking at the results, team “`Protocols`” wins, and you probably should just start using it!